In [109]:
import multiprocessing
import re
from os import path
import pandas as pd
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from nltk.corpus import stopwords
from nltk.corpus import words
import string
import unidecode
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer() 

In [57]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\anas_\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [105]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [59]:
# IMPORT DATA
data = pd.read_csv( 'data/tweets.csv')

# Drop duplicates
data = data.drop_duplicates(subset=['id'])

data = data.reset_index().drop(columns=['index', 'id', 'Unnamed: 0'])
data.head()

#init
ps = PorterStemmer()

In [185]:
def preprocessText(corpus):
    
    # Lower
    corpus = corpus.lower()

    # Remove pictures
    regex = r'pic\.twitter\.com.*'
    corpus.replace('\n','').strip()
    corpus.replace(u'\u2018',"'").replace(u'\u2019',"'")
    corpus= re.sub("https*\S+", " ", corpus)
    corpus = re.sub("(@[A-Za-z0–9]+)",'',corpus)
    
    corpus = re.sub(regex, '', corpus, 0, re.MULTILINE)
    corpus = re.sub(r'[^\w\s]', '', (corpus))
    preprocessed = list()
    stopset = stopwords.words('english') + list(string.punctuation)
    
    # remove stop words and punctuations from string.
    # word_tokenize is used to tokenize the input corpus in word tokens.
    corpus = " ".join([i for i in word_tokenize(corpus) if i not in stopset])
    tok_doc = word_tokenize("".join(corpus))    
    stemmed_doc = [ps.stem(word) for word in tok_doc]
    lemma = [lemmatizer.lemmatize(word) for word in stemmed_doc]
    preprocessed.append(" ".join(lemma))
    
    return preprocessed

def preprocessList(listText):
    preprocessed = list()
    for text in listText:
        preprocList = preprocessText(text)
        preprocessed.append(preprocList)
    return preprocessed

In [216]:
from nltk.tokenize import TweetTokenizer 
tk = TweetTokenizer() 

listText = preprocessList(data.text)
data.loc[:, 'tokens'] = preprocessList(data.text)
sentences = []

for ind in data.index:
    tweet_tokens = tk.tokenize(data['tokens'][ind])
    sentences.append(TaggedDocument(tweet_tokens, [ind]))


print(sentences)

[TaggedDocument(words=['statement'], tags=[0]), TaggedDocument(words=['realli', 'america', 'terribl'], tags=[1]), TaggedDocument(words=['medium', 'establish', 'want', 'race', 'badli', 'never', 'drop', 'race', 'never', 'let', 'support', 'maga'], tags=[2]), TaggedDocument(words=['certainli', 'interest', '24', 'hour'], tags=[3]), TaggedDocument(words=['debat', 'poll', 'look', 'great', 'thank', 'maga', 'americafirst'], tags=[4]), TaggedDocument(words=['say', 'clinton', 'campaign', 'anticathol', 'bigotri'], tags=[5]), TaggedDocument(words=['thank', 'maga', 'americafirst'], tags=[6]), TaggedDocument(words=['cincinnati', 'ohio', 'tomorrow', 'night', '730pm', 'join', 'ohiovotesearli', 'votetrumppence', '16', 'ticket'], tags=[7]), TaggedDocument(words=['littl', 'pickup', 'dishonest', 'medium', 'incred', 'inform', 'provid', 'wikileak', 'dishonest', 'rig', 'system'], tags=[8]), TaggedDocument(words=['thank', 'florida', 'movement', 'never', 'seen', 'never', 'seen', 'let', 'get', 'votetrumppence', 

In [217]:
from nltk.tokenize import TweetTokenizer 
tk = TweetTokenizer() 

def main():
    preprocessed = []
    data.loc[:, 'tokens'] = preprocessList(data.text)
    

    sentences = []
    for ind in data.index:
        tweet_tokens = tk.tokenize(data['tokens'][ind])
        sentences.append(TaggedDocument(tweet_tokens, [ind]))

    #MODEL
    model = Doc2Vec(
        documents=sentences,
        min_count=1,  
        max_vocab_size=None,
        window=50,  # the # of words before and after to be used as context
        size=300,  # is the dimensionality of the feature vector
        workers=multiprocessing.cpu_count(),
        iter=200  # number of iterations (epochs) over the corpus)
    )

    model.save("model")

In [218]:
main()

C:\Users\anas_\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:570: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
C:\Users\anas_\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [265]:
def get_tweet_close(user_text):
    model = Doc2Vec.load("model")
    tokens = preprocessText(user_text)
    tokens = tk.tokenize(tokens[0])
    vector = model.infer_vector(tokens)
    

    result = []
    for tweet_id, confidence in model.docvecs.most_similar([vector], topn=20):
        tweet = data.iloc[tweet_id].to_dict()
        result.append({**tweet, 'confidence': confidence, 'retweet': bool(tweet['retweet'])})

    return result


In [284]:

top20tw = get_tweet_close("Texas")
i = 1
for tweet in top20tw :
    strResult = '<p> Tweet number '+str(i)+' : '+tweet["text"]+'</p>'
    i = i+1
    print(strResult)

<p> Tweet number 1 : Thank you Dallas, Texas! 
https://www.facebook.com/DonaldTrump/posts/10157177470730725:0 …pic.twitter.com/2iYArmM7oN</p>
<p> Tweet number 2 : I will be live tweeting!</p>
<p> Tweet number 3 : Doing Fox & Friends at 7 A.M.</p>
<p> Tweet number 4 : "THE SYSTEM IS RIGGED!"</p>
<p> Tweet number 5 : "@DillonJQuinn: @realDonaldTrump HOW DO I SIGN UP TO BE A CAMPAIGN VOLUNTEER?!"  You will be called!</p>
<p> Tweet number 6 : @DaraghRohan  Happy Birthday and good luck!</p>
<p> Tweet number 7 : Fox and Friends</p>
<p> Tweet number 8 : Fox & Friends at 7.00</p>
<p> Tweet number 9 : Doing a commercial for @NFLONFOX- lots of fun! http://instagram.com/p/eAeLyyGhco/ </p>
<p> Tweet number 10 : .@ewanshearer  Happy Birthday</p>
<p> Tweet number 11 : @VoiceNationLIVE  Work hard and smart!</p>
<p> Tweet number 12 : When will @BarackObama release his transcripts? What is he hiding?</p>
<p> Tweet number 13 : Happy 8th Anniversary to @MELANIATRUMP.pic.twitter.com/y45SSfYx</p>
<p> Tweet